In [1]:
import cv2
import numpy as np
import time
from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment
from filterpy.kalman import KalmanFilter

# --------------------- KalmanBoxTracker ---------------------
class KalmanBoxTracker:
    count = 0

    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([
            [1, 0, 0, 0, 1, 0, 0],
            [0, 1, 0, 0, 0, 1, 0],
            [0, 0, 1, 0, 0, 0, 1],
            [0, 0, 0, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 0, 1]
        ])
        self.kf.H = np.array([
            [1, 0, 0, 0, 0, 0, 0],
            [0, 1, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0]
        ])
        self.kf.R[2:, 2:] *= 10.
        self.kf.P[4:, 4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1, -1] *= 0.01
        self.kf.Q[4:, 4:] *= 0.01

        x1, y1, x2, y2 = bbox
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        s = (x2 - x1) * (y2 - y1)
        r = (x2 - x1) / (y2 - y1)
        self.kf.x[:4] = np.array([[cx], [cy], [s], [r]])

        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        x1, y1, x2, y2 = bbox
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        s = (x2 - x1) * (y2 - y1)
        r = (x2 - x1) / (y2 - y1)
        z = np.array([[cx], [cy], [s], [r]])
        self.kf.update(z)

    def predict(self):
        if (self.kf.x[6] + self.kf.x[2]) <= 0:
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.kf.x)
        return self.kf.x

    def get_state(self):
        x = self.kf.x
        cx, cy, s, r = x[0], x[1], x[2], x[3]
        w = np.sqrt(s * r)
        h = s / w
        x1 = cx - w / 2
        y1 = cy - h / 2
        x2 = cx + w / 2
        y2 = cy + h / 2
        return [x1[0], y1[0], x2[0], y2[0]]

# --------------------- SORT Tracker ---------------------
class Sort:
    def __init__(self, max_age=30, min_hits=2, iou_thresh=0.25):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_thresh = iou_thresh
        self.trackers = []

    def iou(self, bb_test, bb_gt):
        xx1 = np.maximum(bb_test[0], bb_gt[0])
        yy1 = np.maximum(bb_test[1], bb_gt[1])
        xx2 = np.minimum(bb_test[2], bb_gt[2])
        yy2 = np.minimum(bb_test[3], bb_gt[3])
        w = np.maximum(0., xx2 - xx1)
        h = np.maximum(0., yy2 - yy1)
        inter = w * h
        union = ((bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1]) +
                 (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1]) - inter)
        return inter / union

    def update(self, detections):
        trks = np.array([tracker.predict() for tracker in self.trackers])
        trks = np.array([tracker.get_state() for tracker in self.trackers])
        matched, unmatched_dets, unmatched_trks = [], [], []

        if len(trks) > 0 and len(detections) > 0:
            iou_matrix = np.array([[self.iou(det, trk) for trk in trks] for det in detections])
            row_ind, col_ind = linear_sum_assignment(-iou_matrix)

            for r, c in zip(row_ind, col_ind):
                if iou_matrix[r, c] < self.iou_thresh:
                    unmatched_dets.append(r)
                    unmatched_trks.append(c)
                else:
                    matched.append((r, c))

            unmatched_dets += [d for d in range(len(detections)) if d not in row_ind]
            unmatched_trks += [t for t in range(len(trks)) if t not in col_ind]
        else:
            unmatched_dets = list(range(len(detections)))
            unmatched_trks = list(range(len(trks)))

        for t in unmatched_trks:
            self.trackers[t].time_since_update += 1

        for m in matched:
            self.trackers[m[1]].update(detections[m[0]])

        for i in unmatched_dets:
            trk = KalmanBoxTracker(detections[i])
            self.trackers.append(trk)

        results = []
        for trk in self.trackers:
            if trk.time_since_update < 1 and trk.hit_streak >= self.min_hits:
                d = trk.get_state()
                results.append(d + [trk.id])

        self.trackers = [t for t in self.trackers if t.time_since_update <= self.max_age]
        return np.array(results)

# --------------------- Main ---------------------
model = YOLO("yolov8m.pt")  # مدل دقیق‌تر
tracker = Sort()

cap = cv2.VideoCapture("assets/footage/person2.mp4")  # مسیر ویدیو

if not cap.isOpened():
    print("خطا در باز کردن ویدیو")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    start = time.time()

    results = model.predict(frame, verbose=False)[0]
    boxes = []

    for r in results.boxes.data:
        x1, y1, x2, y2, conf, cls = r
        if conf < 0.5:
            continue
        if int(cls) not in [0, 1, 2, 3, 5, 7]:  # فقط اشیای مهم
            continue
        boxes.append([float(x1), float(y1), float(x2), float(y2)])

    boxes = np.array(boxes)
    tracked = tracker.update(boxes)

    for d in tracked:
        x1, y1, x2, y2, track_id = [int(i) for i in d]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    fps = 1 / (time.time() - start + 1e-8)
    cv2.putText(frame, f'FPS: {fps:.1f}', (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 